# holt-winters
https://www.solarwinds.com/blog/holt-winters-forecasting-simplified

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import itertools

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    return 100 / len(y_true) * np.sum(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))
    )

In [3]:
df = pd.read_csv("df_final_bh_amazonas.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

min_year = df.index.year.min()
train_mask = df.index.year < (min_year + 4)
val_mask = df.index.year == (min_year + 4)

y_train = df.loc[train_mask, 'NUMBER_INT']
y_val = df.loc[val_mask, 'NUMBER_INT']

In [4]:
# Fit Holt-Winters model (weekly seasonality, additive) - AKA Triple Exponential Smoothing

trend_opts = ['add', 'mul', None]
seasonal_opts = ['add', 'mul', None]
damped_opts = [True, False]
seasonal_periods = [7, 14]

results = []
result30 = []

In [5]:
for trend, seasonal, damped, period in itertools.product(trend_opts, seasonal_opts, damped_opts, seasonal_periods):
    if trend is None and damped:  # Invalid combo
        continue
    try:
        model = ExponentialSmoothing(
            y_train,
            trend=trend,
            seasonal=seasonal,
            seasonal_periods=period,
            damped_trend=damped
        ).fit(optimized=True)

        forecast = model.forecast(len(y_val))
        mae = mean_absolute_error(y_val, forecast)
        rmse = np.sqrt(mean_squared_error(y_val, forecast))
        r2 = r2_score(y_val, forecast)

        results.append({
            'trend': trend,
            'seasonal': seasonal,
            'damped_trend': damped,
            'seasonal_periods': period,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2
        })
    except Exception as e:
        print(f"Failed: trend={trend}, seasonal={seasonal}, damped={damped}, period={period}: {e}")

/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [6]:
results_df = pd.DataFrame(results)

results_df = results_df.sort_values(by='RMSE')

results_df.head()

,trend,seasonal,damped_trend,seasonal_periods,MAE,RMSE,R2
13,mul,add,True,14,10.663782,12.855305,0.303754
1,add,add,True,14,10.651116,12.864938,0.302710
12,mul,add,True,7,10.647834,12.889968,0.299994
25,None,add,False,14,10.646958,12.930786,0.295554
15,mul,add,False,14,10.634663,12.935209,0.295072


In [7]:
for trend, seasonal, damped, period in itertools.product(trend_opts, seasonal_opts, damped_opts, seasonal_periods):
    if trend is None and damped:  # Invalid combo
        continue
    try:
        model = ExponentialSmoothing(
            y_train,
            trend=trend,
            seasonal=seasonal,
            seasonal_periods=period,
            damped_trend=damped
        ).fit(optimized=True)

        forecast = model.forecast(30)
        mae = mean_absolute_error(y_val[:30], forecast)
        rmse = np.sqrt(mean_squared_error(y_val[:30], forecast))
        r2 = r2_score(y_val[:30], forecast)

        result30.append({
            'trend': trend,
            'seasonal': seasonal,
            'damped_trend': damped,
            'seasonal_periods': period,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2
        })
    except Exception as e:
        print(f"Failed: trend={trend}, seasonal={seasonal}, damped={damped}, period={period}: {e}")

/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/correia/Documents/Projects/hospital-adm-pw-rel/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [8]:
result30 = pd.DataFrame(result30)

result30 = result30.sort_values(by='RMSE')

result30.head()

,trend,seasonal,damped_trend,seasonal_periods,MAE,RMSE,R2
18,mul,mul,False,7,10.562390,12.182123,-1.216420
6,add,mul,False,7,11.319277,12.865021,-1.471879
26,None,mul,False,7,11.370499,12.912035,-1.489978
14,mul,add,False,7,11.469571,12.915994,-1.491506
4,add,mul,True,7,11.440035,12.981796,-1.516957


# OLD

In [9]:
# # Forecast for validation period (1 year)
# forecast_hw = hw_model.forecast(len(y_val))

In [10]:
# mse = mean_squared_error(y_val, forecast_hw)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_val, forecast_hw)
# mape_val = mape(y_val, forecast_hw)
# smape_val = smape(y_val, forecast_hw)
# r2 = r2_score(y_val, forecast_hw)
# corr = np.corrcoef(y_val, forecast_hw)[0, 1]

# print("Holt-Winters Evaluation - Full Year:")
# print(f"MAE:   {mae:.2f}")
# print(f"RMSE:  {rmse:.2f}")
# print(f"MAPE:  {mape_val:.2f}%")
# print(f"SMAPE: {smape_val:.2f}%")
# print(f"R^2:   {r2:.2f}")
# print(f"Corr:  {corr:.2f}")

In [11]:
# # Forecast for first 30 days
# forecast_30 = hw_model.forecast(30)
# y_val_30 = y_val[:30]

# # Evaluation metrics - 30 Days
# mse_30 = mean_squared_error(y_val_30, forecast_30)
# rmse_30 = np.sqrt(mse_30)
# mae_30 = mean_absolute_error(y_val_30, forecast_30)
# mape_val_30 = mape(y_val_30, forecast_30)
# smape_val_30 = smape(y_val_30, forecast_30)
# r2_30 = r2_score(y_val_30, forecast_30)
# corr_30 = np.corrcoef(y_val_30, forecast_30)[0, 1]

# print("\nHolt-Winters Evaluation - First 30 Days:")
# print(f"MAE:   {mae_30:.2f}")
# print(f"RMSE:  {rmse_30:.2f}")
# print(f"MAPE:  {mape_val_30:.2f}%")
# print(f"SMAPE: {smape_val_30:.2f}%")
# print(f"R^2:   {r2_30:.2f}")
# print(f"Corr:  {corr_30:.2f}")

In [12]:
# # Plot forecast vs validation (full year)
# plt.figure(figsize=(14, 5))
# plt.plot(y_val.index, y_val, label="Validation", color="blue")
# plt.plot(y_val.index, forecast_hw, label="Holt-Winters Forecast", color="orange")
# plt.title("Holt-Winters Forecast vs Validation (Full Year)")
# plt.xlabel("Date")
# plt.ylabel("NUMBER_INT")
# plt.legend()
# plt.tight_layout()
# plt.show()

In [13]:
# # Plot forecast vs validation (first 30 days)
# plt.figure(figsize=(14, 5))
# plt.plot(y_val_30.index, y_val_30, label="Actual", color="blue")
# plt.plot(y_val_30.index, forecast_30, label="Forecast (30 Days)", color="orange")
# plt.title("Holt-Winters Forecast vs Validation (First 30 Days)")
# plt.xlabel("Date")
# plt.ylabel("NUMBER_INT")
# plt.legend()
# plt.tight_layout()
# plt.show()

In [14]:
# residuals = y_val - forecast_hw
# plt.figure(figsize=(12, 4))
# plt.plot(residuals)
# plt.title("Residuals of Holt-Winters Forecast")
# plt.xlabel("Date")
# plt.ylabel("Residuals")
# plt.tight_layout()
# plt.show()


In [15]:
# from statsmodels.graphics.tsaplots import plot_acf

# plot_acf(residuals, lags=30)
# plt.title("ACF of Residuals")
# plt.show()
